In [1]:
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor  
import random
from matplotlib import pyplot as plt

In [2]:
train_df=pd.read_csv('../input/amazonml/train.csv')

In [3]:
test_df=pd.read_csv('../input/amazonml/test.csv')

In [4]:
train_df.head()

,customer_id,customer_visit_score,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
0,csid_1,13.168425,9.447662,-0.070203,-0.139541,0.436956,4.705761,2.537985,7.959503,C,F,0
1,csid_2,17.092979,7.329056,0.153298,-0.102726,0.380340,4.205138,4.193444,17.517381,C,A,0
2,csid_3,17.505334,5.143676,0.106709,0.262834,0.417648,4.479070,3.878971,12.595155,C,BA,0
3,csid_4,31.423381,4.917740,-0.020226,-0.100526,0.778130,5.055535,2.708940,4.795073,AA,F,0
4,csid_5,11.909502,4.237073,0.187178,0.172891,0.162067,3.445247,3.677360,56.636326,C,AA,0


In [5]:
test_df.head()

,customer_id,customer_visit_score,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1
0,csid_10739,22.500174,3.702145,0.118811,0.093253,0.689237,5.786842,3.748009,7.661716,B,BA
1,csid_10740,13.225950,3.600102,0.130985,0.053575,0.142756,3.940359,3.469659,33.999679,B,AA
2,csid_10741,14.603376,5.084047,-0.004651,-0.036299,0.333709,4.254662,3.855939,33.485630,B,AA
3,csid_10742,26.149905,3.025501,0.229740,0.204591,3.768405,6.942625,2.362421,0.795372,B,BA
4,csid_10743,19.810630,5.020400,-0.011347,-0.028780,0.286841,4.585513,3.278624,15.052058,C,BA


In [6]:
train_df.duplicated().sum()

0

In [7]:
train_df['customer_id'].duplicated().sum()

0

In [8]:
train_df['customer_category'].unique()

array([0, 1])

In [9]:
train_df.skew()

customer_visit_score                0.101448
customer_product_search_score       0.289272
customer_ctr_score                  3.216021
customer_stay_score                 5.008726
customer_frequency_score            4.083013
customer_product_variation_score    1.851647
customer_order_score                0.576865
customer_affinity_score             2.993484
customer_category                   2.330354
dtype: float64

In [10]:
train_df.corr(method='spearman')['customer_category']

customer_visit_score               -0.414148
customer_product_search_score      -0.291749
customer_ctr_score                  0.531189
customer_stay_score                 0.497748
customer_frequency_score            0.431105
customer_product_variation_score    0.433911
customer_order_score               -0.407798
customer_affinity_score            -0.427537
customer_category                   1.000000
Name: customer_category, dtype: float64

In [11]:
train_df.isnull().sum()

customer_id                          0
customer_visit_score                 0
customer_product_search_score       42
customer_ctr_score                   0
customer_stay_score                 37
customer_frequency_score             0
customer_product_variation_score    46
customer_order_score                66
customer_affinity_score              0
customer_active_segment             23
X1                                  37
customer_category                    0
dtype: int64

In [12]:
train_df[['customer_product_search_score','customer_stay_score','customer_product_variation_score','customer_order_score', 'customer_active_segment','X1']] = train_df[['customer_product_search_score','customer_stay_score','customer_product_variation_score','customer_order_score', 'customer_active_segment','X1']].ffill()
# train_df.dropna(how='any', inplace=True)    

In [13]:
train_df.head()

,customer_id,customer_visit_score,customer_product_search_score,customer_ctr_score,customer_stay_score,customer_frequency_score,customer_product_variation_score,customer_order_score,customer_affinity_score,customer_active_segment,X1,customer_category
0,csid_1,13.168425,9.447662,-0.070203,-0.139541,0.436956,4.705761,2.537985,7.959503,C,F,0
1,csid_2,17.092979,7.329056,0.153298,-0.102726,0.380340,4.205138,4.193444,17.517381,C,A,0
2,csid_3,17.505334,5.143676,0.106709,0.262834,0.417648,4.479070,3.878971,12.595155,C,BA,0
3,csid_4,31.423381,4.917740,-0.020226,-0.100526,0.778130,5.055535,2.708940,4.795073,AA,F,0
4,csid_5,11.909502,4.237073,0.187178,0.172891,0.162067,3.445247,3.677360,56.636326,C,AA,0


In [14]:
target = train_df['customer_category']
train_df.drop('customer_category', inplace=True, axis=1)
train_df.drop('customer_id', inplace=True, axis=1)

In [15]:
train_df.drop('customer_active_segment', inplace=True, axis=1)
train_df.drop('X1', inplace=True, axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.3, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
mdl = LogisticRegression()

In [19]:
mdl.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [20]:
y_predicted = mdl.predict(X_test)

In [21]:
score = metrics.precision_score(y_test, y_predicted, average='macro')

In [22]:
print('RMSE:', score)

RMSE: 0.971102608719248


In [23]:
#test df treatment
test_df.isnull().sum()

customer_id                          0
customer_visit_score                 0
customer_product_search_score       29
customer_ctr_score                   0
customer_stay_score                 16
customer_frequency_score             0
customer_product_variation_score    43
customer_order_score                41
customer_affinity_score              0
customer_active_segment             12
X1                                  25
dtype: int64

In [24]:
test_df[['customer_product_search_score','customer_stay_score','customer_product_variation_score','customer_order_score', 'customer_active_segment','X1']] = test_df[['customer_product_search_score','customer_stay_score','customer_product_variation_score','customer_order_score', 'customer_active_segment','X1']].ffill()

In [25]:
customer_id = test_df['customer_id']

In [26]:
test_df.drop('customer_id', inplace=True, axis=1)
test_df.drop('customer_active_segment', inplace=True, axis=1)
test_df.drop('X1', inplace=True, axis=1)

In [27]:
output = mdl.predict(test_df)

In [28]:
# intialise data of lists. 
result_df = {'customer_id':customer_id, 'customer_category':output} 
  
# Create DataFrame 
df = pd.DataFrame(result_df) 

In [29]:
df.to_csv('Results.csv', index=False)